# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
#----------------------------------------------------------------------------------------
# DEPENDANCIES
#----------------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#----------------------------------------------------------------------------------------
# IMPORT API KEY
#----------------------------------------------------------------------------------------
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
#----------------------------------------------------------------------------------------
# READ CITIES CSV FILE
#----------------------------------------------------------------------------------------
weather_data_path = "../WeatherPy/cities_df.csv"
weather_data = pd.read_csv(weather_data_path)

#----------------------------------------------------------------------------------------
# DROP UNNAMED COLUMN
#----------------------------------------------------------------------------------------
weather_data = weather_data[['City', 'Country', 'Latitude', 'Longitude', 'Temperature',
       'Humidity', 'Cloudiness', 'Wind_Speed']]

#----------------------------------------------------------------------------------------
# PREVIEW DATAFRAME
#----------------------------------------------------------------------------------------
weather_data.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
0,Inazawa,JP,35.2500,136.7833,25.34,37,40,3.16
1,Kalavai,IN,12.7703,79.4200,36.75,30,1,4.26
2,Palaiókipos,GR,39.0558,26.4511,16.57,67,0,0.00
3,Mananjary,MG,-21.2167,48.3333,24.14,81,88,2.26
4,Langholm,GB,55.1510,-2.9989,5.73,91,51,2.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#----------------------------------------------------------------------------------------
# STORE REQUIRED LAT/LONG AND HUMIDITY DATA IN VARIABLES
#----------------------------------------------------------------------------------------
locations = weather_data[["Latitude", "Longitude"]]
humidity = weather_data["Humidity"]

In [13]:
#----------------------------------------------------------------------------------------
# CREATE HEATMAP
#----------------------------------------------------------------------------------------
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
                                 
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#----------------------------------------------------------------------------------------------------
# FIND CITIES WITH PERFECT CONDITIONS. DROP ANY ROWS THAT DON"T CONTAIN ALL THREE CONDITIONS BELOW
#--- TEMPERATURE BETWEEN 21 & 27 DEGREES CENTIGRADE
#--- WIND SPEED LESS THAN 10 MPH
#--- ZERO CLOUDINESS
#----------------------------------------------------------------------------------------------------

# Filter the dataframe
perfect_weather_df = weather_data[(weather_data['Temperature'] >= 21) & (weather_data['Temperature'] < 26) & (weather_data['Wind_Speed'] < 10) & (weather_data['Cloudiness'] == 0)]

# Visualize
print(len(perfect_weather_df))
perfect_weather_df.head()


11


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
68,Les Abymes,GP,16.2710,-61.5045,22.62,94,0,0.00
86,San Nicolás de los Garza,MX,25.7500,-100.3000,25.03,76,0,5.14
108,Jeffersonville,US,38.2776,-85.7372,24.05,60,0,6.17
122,Pedro Leopoldo,BR,-19.6181,-44.0431,22.76,78,0,4.63
166,Córdoba,AR,-31.4135,-64.1811,23.44,73,0,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
#----------------------------------------------------------------------------------------
# STORE CITIES WITH DESIRED WEATHER INTO VARIABLE NAMED 'HOTEL_DF' AND ADD NEW COLUMN
#----------------------------------------------------------------------------------------
hotel_df = perfect_weather_df
hotel_df["Hotel Name"] = ''
hotel_df


C:\Users\rjtur\AppData\Local\Temp/ipykernel_280/2423287130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name
68,Les Abymes,GP,16.2710,-61.5045,22.62,94,0,0.00,
86,San Nicolás de los Garza,MX,25.7500,-100.3000,25.03,76,0,5.14,
108,Jeffersonville,US,38.2776,-85.7372,24.05,60,0,6.17,
122,Pedro Leopoldo,BR,-19.6181,-44.0431,22.76,78,0,4.63,
166,Córdoba,AR,-31.4135,-64.1811,23.44,73,0,3.60,
333,Guadalupe Victoria,MX,32.2869,-115.1050,21.04,10,0,3.86,
402,Siavonga,ZM,-16.5382,28.7088,23.04,70,0,1.15,
404,Kettering,US,39.6895,-84.1688,21.96,64,0,5.66,
511,Ascension,US,30.2169,-90.9001,23.00,97,0,2.78,
520,Mersin Province,TR,36.8620,34.6509,23.04,52,0,2.20,


In [20]:
#----------------------------------------------------------------------------------------
# SET PARAMETERS TO SEARCH FOR HOTELS WITHIN 5000 METERS
# HIT THE GOOGLE PLACES API FOR EACH CITY"S COORDINATES
# STORE THE FIRST HOTEL RESULT INTO THE DATAFRAME
#----------------------------------------------------------------------------------------

# Empty list to store hotel names
hotel_list = []

# Set list of city names for loop
cities_data = hotel_df["City"]

# Loop through each city in the list of desired cities
for city in cities_data:

    # Set latitude and longitude values
    lat = hotel_df[hotel_df["City"] == city]["Latitude"].values[0]
    long = hotel_df[hotel_df["City"] == city]["Longitude"].values[0]

    # Set parameters
    target_coordinates = str(lat) + ", " + str(long)
    target_radius = 5000
    target_search = 'hotel'
    target_type = 'lodging'

    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # convert response to json
    hotels_data = response.json()   

    # Append hotel name to hotel_list
    hotel_list.append(hotels_data["results"][0]["name"])

# Add hotel names to hotel_df    
hotel_df["Hotel Name"] = hotel_list

# Display df
hotel_df

C:\Users\rjtur\AppData\Local\Temp/ipykernel_280/3868740856.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel_list


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name
68,Les Abymes,GP,16.2710,-61.5045,22.62,94,0,0.00,Établissement Bristol
86,San Nicolás de los Garza,MX,25.7500,-100.3000,25.03,76,0,5.14,"Holiday Inn Monterrey Norte, an IHG Hotel"
108,Jeffersonville,US,38.2776,-85.7372,24.05,60,0,6.17,"The Galt House Hotel, Trademark Collection by ..."
122,Pedro Leopoldo,BR,-19.6181,-44.0431,22.76,78,0,4.63,JJ Imoveis PL
166,Córdoba,AR,-31.4135,-64.1811,23.44,73,0,3.60,Hotel Sussex
333,Guadalupe Victoria,MX,32.2869,-115.1050,21.04,10,0,3.86,Hotel Dial
402,Siavonga,ZM,-16.5382,28.7088,23.04,70,0,1.15,Lavongora Waterfront Lodge
404,Kettering,US,39.6895,-84.1688,21.96,64,0,5.66,Budget Inn
511,Ascension,US,30.2169,-90.9001,23.00,97,0,2.78,Highland Inn
520,Mersin Province,TR,36.8620,34.6509,23.04,52,0,2.20,Nostalgia Boutique Hotel


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [22]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))